In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os

In [ ]:
if not os.path.exists('./datalake/my_folder'):
   os.makedirs('./datalake/my_folder')

In [ ]:
# chunk_size = 100000  # Adjust the chunk size as per your system’s memory constraints
# csv_file = url+'data/ml-latest/movies.csv'

# df = pd.read_csv(csv_file)


# set_genres = set()
# for d in df['genres']:
#   for stri in d.split('|'):
#     set_genres.add(stri)
# df_set = pd.DataFrame(list(set_genres))
# df_set.to_csv(url + 'data/ml-latest/genres.csv')



Crawl all Genre

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
# Địa chỉ URL cần lấy dữ liệu
urlMovie = 'https://phimmoiiii.net/'

# Lấy nội dung HTML từ URL
response = requests.get(urlMovie)
html_content = response.content

# Phân tích HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Tìm tất cả các thẻ ul có class là 'sub-menu'
sub_menu_ul = soup.find('ul', class_='sub-menu')

list_genre = []

# Nếu tìm thấy thẻ ul có class là 'sub-menu'
if sub_menu_ul:
    # Tìm tất cả các thẻ li trong thẻ ul 'sub-menu'
    sub_menu_items = sub_menu_ul.find_all('li')

    # Duyệt qua từng thẻ li
    for item in sub_menu_items:
        # Tìm thẻ a trong thẻ li
        a_tag = item.find('a')
        # Nếu tìm thấy thẻ a
        if a_tag:
            # Trích xuất href và nội dung của thẻ a
            href = a_tag.get('href')
            content = a_tag.text.strip()
            list_genre.append([href, content])
            print(f"Link: {href}, Content: {content}")
else:
    print("Không tìm thấy thẻ ul có class là 'sub-menu'")

df_genre = pd.DataFrame(list_genre, columns=['url', 'name'])
df_genre.to_csv('./datalake/genre_vietnamese.csv',index=False)




Crawle movie by year

In [ ]:
def crawlMovieByPage(page, url, year):
    print(f'[START] Page {page}, year {year}: ')
    list_movies = []
    url_movie = url + '/{0}'.format(year) if page == 1 else url + '/{0}/page/{1}'.format(year, page)
    # Lấy nội dung HTML từ URL
    try:
      response = requests.get(url_movie, timeout=10)
    except requests.exceptions.Timeout:
      print("Timed out")
      return False
    if response.status_code > 300:
       return False
    html_content = response.content

    # Phân tích HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Tìm tất cả các thẻ div có class là 'items'
    list_movie_div = soup.find('div', class_='items')

    # Nếu tìm thấy thẻ ul có class là 'sub-menu'
    if list_movie_div:
        # Tìm tất cả các thẻ li trong thẻ ul 'sub-menu'
        movie_items = list_movie_div.find_all('article')

        # Duyệt qua từng thẻ li
        for item in movie_items:
            mv_img_url = ''
            mv_vn_name = ''
            mv_other_name = ''
            mv_url = ''
            mv_status = ''

            try:
              img_tag = item.find('img')
              status_tag = item.find('div', class_='trangthai')
              url_mv_div = item.select('div.data h3 a')[0]
              name_or_div = item.select('div.data span span')[0]
              # print(img_tag)
              # print(status_tag)
              # print(url_mv_div)
              # print(name_en_div)

              if img_tag:
                mv_img_url = img_tag.get('src')
              if status_tag:
                mv_status = status_tag.text.strip()
              if url_mv_div:
                mv_vn_name = url_mv_div.text.strip()
                mv_url = url_mv_div.get('href')
              if name_or_div:
                mv_other_name = name_or_div.text.strip()
              print('  ', mv_vn_name, mv_other_name, mv_url, mv_img_url, mv_status)
              list_movies.append([mv_vn_name, mv_other_name, mv_url, mv_img_url, mv_status])
            except Exception as error:
              print('[*] Error', error)
              return False
    if len(list_movies) > 0:
      df_movies = pd.DataFrame(list_movies, columns=['name_vn', 'name_other', 'url','image','status'])
      df_movies.index.name='id'
      if not os.path.exists(f'./datalake/{year}'):
        os.makedirs(f'./datalake/{year}')
      df_movies.to_csv('./datalake/{year}/movie_page_{page}.csv'.format(year=year, page=page))
      return True
    return False

In [ ]:
year = 1990

for i in range(1, 300):
    if i >= 1:
      is_success = crawlMovieByPage(i,'https://phimmoiiii.net/nam-phat-hanh',year)
      if not(is_success):
        print(f'Finish year {year}')
        time.sleep(1)
        break

In [ ]:
year = 1990

while year <= 2024:
  for i in range(1, 300):
    if i >= 1:
      is_success = crawlMovieByPage(i,'https://phimmoiiii.net/nam-phat-hanh',year)
      if not(is_success):
        print(f'Finish year {year}')
        time.sleep(1)
        break
  year += 1



In [ ]:
def getUrlVideoOfMovie(id):
  url_ajax = 'https://phimmoiiii.net/wp-admin/admin-ajax.php'
  payload = {'action': 'doo_player_ajax','type': 'movie','post': id, 'nume': 1 }
  responseAjax = requests.post(url_ajax,data= payload)

  if responseAjax.status_code == 200:
    response_data = responseAjax.json()
    embed_url = response_data.get('embed_url')
    if embed_url:
        return embed_url
  else:
    print('[ERROR] Fail to get the url video of movie id {}'.format(id))
    return ''

In [ ]:
# Crawl each movie by url

def crawlMovieByUrl(url_movie):
    # Lấy nội dung HTML từ URL
    response = requests.get(url_movie)
    html_content = response.content

    movie_id = -1
    movie_video_url = ''
    movie_description = ''
    movie_genres = []
    movie_directors = []
    movie_actors = []
    movie_country = ''
    movie_length = 0
    movie_date = ''

    # Phân tích HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Data id and url of movie and video movie
    input_post_id = soup.find(attrs={'name': 'postid'})
    if input_post_id:
      movie_id = int(input_post_id.get('value'))
      movie_video_url = getUrlVideoOfMovie(movie_id)

    # Data date
    span_date = soup.find('span', attrs={'itemprop': 'dateCreated'})
    if span_date:
      movie_date = span_date.text.strip()

    # Data time
    span_time = soup.find('span', attrs={'itemprop': 'duration'})
    if span_time:
      movie_length = span_time.text.strip()

    # Data country
    span_country = soup.find('span', attrs={'class': 'country'})
    if span_country:
      movie_country = span_country.text.strip()

    # Data description
    span_description = soup.find('div', attrs={'itemprop': 'description'})
    if span_description:
      p_description = span_description.find('p')
      if p_description:
        movie_description = p_description.text.strip()

    # Div genres
    div_genres = soup.find('div', class_='sgeneros')
    if div_genres:
      list_genres_a = div_genres.find_all('a')
      if(len(list_genres_a) > 0):
        for a in list_genres_a:
          movie_genres.append(a.text)

    # Div directors
    div_directors = soup.find_all('div', class_='person', attrs={'itemprop': 'director'})
    if div_directors and len(div_directors) > 0:
      for div_director in div_directors:
        director_img = div_director.find('img')
        director_data = div_director.find('a', attrs={'itemprop': 'url'})
        director_img_url = ''
        director_name = ''
        director_type = div_director.get('itemtype')
        if director_img:
          director_img_url = director_img.get('href')
        if director_data:
          director_name = director_data.text
        movie_directors.append({
            'type': director_type,
            'name': director_name,
            'image': director_img_url
        })
    # Div actors
    div_actors = soup.find_all('div', class_='person', attrs={'itemprop': 'actor'})
    if div_actors and len(div_actors) > 0:
      for div_actor in div_actors:
        actor_img_url = ''
        actor_name = ''
        actor_character = ''
        actor_type = div_actor.get('itemtype')
        actor_img = div_actor.find('img')
        div_actor_character = div_actor.find('div', class_='caracter')
        div_actor_data = div_actor.find('a', attrs={'itemprop': 'url'})
        if actor_img:
          actor_img_url = actor_img.get('src')
        if div_actor_data:
          actor_name = div_actor_data.text
        if div_actor_character:
          actor_character = div_actor_character.text
        movie_actors.append({
            'type': actor_type,
            'name': actor_name,
            'image': actor_img_url,
            'character': actor_character
        })
    # stringFormat = '[Data]: id: %d, img: %s, country: %s, length: %s, date: %s'
    # print(stringFormat.format(movie_id, movie_video_url, movie_country, movie_length, movie_date))
    # print(movie_description)
    # print(movie_genres)
    # print(movie_directors)
    # print(movie_actors)

    result = {
        'movie_id': movie_id,
        'movie_video': movie_video_url,
        'movie_country': movie_country,
        'movie_length': movie_length,
        'movie_date': movie_date,
        'movie_description': movie_description,
        'movie_genres': movie_genres,
        'movie_directors': movie_directors,
        'movie_actors': movie_actors
    }

    return result

test = crawlMovieByUrl('https://phimmoiiii.net/phim-le/quy-co-mang-nhen')
print(test)
